In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb

# Load Data

In [2]:
path = ''

In [3]:
#Load Train
train = pd.read_csv(path + 'train.csv')
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
#Load Test
test = pd.read_csv(path + 'test.csv')
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


# Visualize and Feature Engineer

In [5]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479
GarageFinish,81,0.055479


In [6]:
#Split the train into x_train and y_train so that SalePrice can be kept separate for training later
y = train.SalePrice
X = train.drop(columns=["SalePrice"], axis=1)

In [7]:
X['Type'] = 'train'
test['Type'] = 'test'
#test['SalePrice'] = -1
data = X.append(test)

In [8]:
columns_having_null_values = data[data.columns[data.isnull().sum()>0]]
columns_having_null_values

,MSZoning,LotFrontage,Alley,Utilities,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,...,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PoolQC,Fence,MiscFeature,SaleType
0,RL,65.0,NaN,AllPub,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,...,2003.0,RFn,2.0,548.0,TA,TA,NaN,NaN,NaN,WD
1,RL,80.0,NaN,AllPub,MetalSd,MetalSd,None,0.0,Gd,TA,...,1976.0,RFn,2.0,460.0,TA,TA,NaN,NaN,NaN,WD
2,RL,68.0,NaN,AllPub,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,...,2001.0,RFn,2.0,608.0,TA,TA,NaN,NaN,NaN,WD
3,RL,60.0,NaN,AllPub,Wd Sdng,Wd Shng,None,0.0,TA,Gd,...,1998.0,Unf,3.0,642.0,TA,TA,NaN,NaN,NaN,WD
4,RL,84.0,NaN,AllPub,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,...,2000.0,RFn,3.0,836.0,TA,TA,NaN,NaN,NaN,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,21.0,NaN,AllPub,CemntBd,CmentBd,None,0.0,TA,TA,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,WD
1455,RM,21.0,NaN,AllPub,CemntBd,CmentBd,None,0.0,TA,TA,...,1970.0,Unf,1.0,286.0,TA,TA,NaN,NaN,NaN,WD
1456,RL,160.0,NaN,AllPub,VinylSd,VinylSd,None,0.0,TA,TA,...,1960.0,Unf,2.0,576.0,TA,TA,NaN,NaN,NaN,WD
1457,RL,62.0,NaN,AllPub,HdBoard,Wd Shng,None,0.0,Gd,TA,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,MnPrv,Shed,WD


In [9]:
# We have to check what values are there in the table so that we can fill values according to real world scenario.
data['Electrical'].value_counts()

SBrkr    2671
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64

In [10]:
data['MSZoning'].value_counts()
#Filling null values with 'RL'
data['MSZoning'].fillna("RL",inplace=True)
data['Electrical'].fillna("Sbrkr", inplace=True)
#Filling nul values with mean
data['LotFrontage'].fillna(data['LotFrontage'].mean(), inplace=True)

data['Alley'].fillna("Nothing", inplace=True)
data['Utilities'].fillna("AllPub", inplace=True)
data['Exterior1st'].fillna("VinylSd", inplace=True)
data['Exterior2nd'].fillna("VinylSd", inplace=True)
data['MasVnrArea'].fillna(0, inplace=True)
data['MasVnrType'].fillna("None", inplace=True)
data['BsmtCond'].fillna("No", inplace=True)
data['BsmtExposure'].fillna("NB", inplace=True)
data['BsmtFinType1'].fillna("NB", inplace=True)
data['BsmtFinSF1'].fillna(0.0, inplace=True)
data['BsmtFinSF2'].fillna(0.0, inplace=True)
data['BsmtUnfSF'].fillna(0.0, inplace=True)
data['TotalBsmtSF'].fillna(0.0, inplace=True)
data['BsmtFullBath'].fillna(0.0, inplace=True)
data['BsmtHalfBath'].fillna(0.0, inplace=True)
data['KitchenQual'].fillna("TA", inplace=True)
data['Functional'].fillna("Typ", inplace=True)
data['FireplaceQu'].fillna("None", inplace=True)
data['GarageType'].fillna("No", inplace=True)
data['GarageYrBlt'].fillna(0, inplace=True)
data['GarageFinish'].fillna("No", inplace=True)
data['GarageCars'].fillna(0, inplace=True)
data['GarageArea'].fillna(0, inplace=True)
data['GarageQual'].fillna("No", inplace=True)
data['GarageCond'].fillna("No", inplace=True)
data['PoolQC'].fillna("No", inplace=True)
data['Fence'].fillna("No", inplace=True)
data['MiscFeature'].fillna("No", inplace=True)
data['SaleType'].fillna("Con", inplace=True)
data['SaleCondition'].fillna("None", inplace=True)
data['BsmtQual'].fillna("TA", inplace=True)
data['BsmtFinType2'].fillna("Unf", inplace=True)

In [11]:
data.isnull().sum().sum()

0

In [12]:
int_columns = data[data.columns[data.dtypes=='int']]
int_columns.columns

Index([], dtype='object')

In [13]:
data['MSZoning'].unique()

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

In [14]:
float_columns = data[data.columns[data.dtypes=='float']]
float_columns.columns

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')

In [15]:
object_columnns = data[data.columns[data.dtypes=='object']]
object_columnns.columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', 'Type'],
      dtype='object')

In [16]:
from sklearn.preprocessing import LabelEncoder
for i in object_columnns:
    label = LabelEncoder()
    label.fit(data[i].values)
    data[i] = label.transform(data[i].values)

In [17]:
int_columns = data[data.columns[data.dtypes=='int']]
int_columns.columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', 'Type'],
      dtype='object')

In [18]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Type
0,1,60,3,65.0,8450,1,1,3,3,0,...,0,3,4,1,0,2,2008,8,4,1
1,2,20,3,80.0,9600,1,1,3,3,0,...,0,3,4,1,0,5,2007,8,4,1
2,3,60,3,68.0,11250,1,1,0,3,0,...,0,3,4,1,0,9,2008,8,4,1
3,4,70,3,60.0,9550,1,1,0,3,0,...,0,3,4,1,0,2,2006,8,0,1
4,5,60,3,84.0,14260,1,1,0,3,0,...,0,3,4,1,0,12,2008,8,4,1


In [19]:
X_ = data[data.Type==1]
X_ = X_.drop(["Type"], axis=1)
test_ = data[data.Type==0]
test_ = test_.drop(["Type"], axis=1)
X_.shape, y.shape, test_.shape 

((1460, 80), (1460,), (1459, 80))

# Split Data


In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_, y, test_size=0.1, random_state=19520197)


## Basic Model

SVR


In [21]:
from sklearn.metrics import r2_score


In [22]:
svr = svm.SVR()
svr.fit(x_train,y_train)
score_svr = r2_score(svr.predict(x_test), y_test)
score_test = [score_svr]
model = ['model_svr']
print(score_svr)

-3560345.0741296033


Random Forest


In [23]:
model_randomforest = RandomForestRegressor()
model_randomforest.fit(x_train, y_train)
print(model_randomforest.score(x_test, y_test))
score_test.append(model_randomforest.score(x_test, y_test))
model.append("model_randomforest")

0.8965331233510274


XGBoost


In [24]:
model_xgboost = xgb.XGBRegressor()
model_xgboost.fit(x_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [25]:
xgb_score = model_xgboost.score(x_test, y_test)
score_test.append(xgb_score)
model.append("model_xgboost")
xgb_score


0.9176856990451383

LightGBM


In [26]:
lightgbm = lgb.LGBMRegressor(objective='regression')
lightgbm.fit(x_train, y_train)

LGBMRegressor(objective='regression')

In [27]:
score_test.append(lightgbm.score(x_test, y_test))
model.append("model_lightgbm")
lightgbm.score(x_test, y_test)

0.9223148562090775

Scoreboard


In [28]:
final_scores = pd.DataFrame()
final_scores['model_name'] = model
final_scores['score_test'] = score_test
final_scores = final_scores.sort_values(by=['score_test'], ascending=0)
final_scores

,model_name,score_test
3,model_lightgbm,9.223149e-01
2,model_xgboost,9.176857e-01
1,model_randomforest,8.965331e-01
0,model_svr,-3.560345e+06


# Optuna


In [29]:
import optuna 
from optuna import Trial

c:\Users\thien\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Randomforest


In [30]:
def rdf_reg_objective(trial: Trial):

    n_estimators = trial.suggest_int('n_estimators', 10, 300, 10)
    max_depth = trial.suggest_int('max_depth', 3, 50, step=1)
    min_samples_split = trial.suggest_int('min_samples_split',2,20, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20, step =1 )
    max_features = trial.suggest_float('max_features', 0.1, 1, step=0.05)

    rdf_reg = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_features=max_features, n_jobs=-1, random_state=42)
    cv = KFold(n_splits=5, shuffle=True)
    scores = cross_val_score(rdf_reg, x_train, y_train, cv=cv, scoring='r2')
    return scores.mean()

rdf_study = optuna.create_study(sampler=optuna.samplers.RandomSampler(), direction='maximize')
rdf_study.optimize(rdf_reg_objective, n_trials=300)

[I 2022-05-22 03:13:47,046] A new study created in memory with name: no-name-a02cd632-bdb6-42d3-9c4a-396781dd6d03
[I 2022-05-22 03:13:49,943] Trial 0 finished with value: 0.8125185692430538 and parameters: {'n_estimators': 250, 'max_depth': 44, 'min_samples_split': 18, 'min_samples_leaf': 18, 'max_features': 0.25}. Best is trial 0 with value: 0.8125185692430538.
[I 2022-05-22 03:13:50,797] Trial 1 finished with value: 0.8115899781351498 and parameters: {'n_estimators': 30, 'max_depth': 14, 'min_samples_split': 7, 'min_samples_leaf': 16, 'max_features': 0.1}. Best is trial 0 with value: 0.8125185692430538.
[I 2022-05-22 03:13:52,782] Trial 2 finished with value: 0.83315877903444 and parameters: {'n_estimators': 220, 'max_depth': 22, 'min_samples_split': 12, 'min_samples_leaf': 1, 'max_features': 0.6}. Best is trial 2 with value: 0.83315877903444.
[I 2022-05-22 03:13:54,548] Trial 3 finished with value: 0.8267873620044977 and parameters: {'n_estimators': 200, 'max_depth': 7, 'min_samples

In [31]:
# get params and re-train
rdf_params = rdf_study.best_params
opt_rdf_reg = RandomForestRegressor(n_estimators = rdf_params['n_estimators'],
                                    max_depth = rdf_params['max_depth'],
                                    min_samples_split = rdf_params['min_samples_split'],
                                    min_samples_leaf = rdf_params['min_samples_leaf'],
                                    max_features = rdf_params['max_features'])
opt_rdf_reg.fit(x_train, y_train)
opt_rdf_reg.score(x_test, y_test)


0.9360139323846859

XGBoost


In [32]:
def xgb_reg_objective(trial: Trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 300, 10)
    max_depth = trial.suggest_int('max_depth', 3, 8, step=1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 0.9, step=0.05)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-2, 1)

    xgb_reg = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, colsample_bytree=colsample_bytree, learning_rate=learning_rate, n_jobs=-1, random_state=42)
    cv = KFold(n_splits=5, shuffle=True)
    scores = cross_val_score(xgb_reg, x_train, y_train, cv=cv, scoring='r2')
    return scores.mean()

xgb_study = optuna.create_study(sampler=optuna.samplers.RandomSampler(), direction='maximize')
xgb_study.optimize(xgb_reg_objective, n_trials=300)

[I 2022-05-22 03:21:06,193] A new study created in memory with name: no-name-fe57d5e8-7b64-4302-9fd9-e57f816a1afd
[I 2022-05-22 03:21:06,906] Trial 0 finished with value: 0.802536977905608 and parameters: {'n_estimators': 220, 'max_depth': 3, 'colsample_bytree': 0.75, 'learning_rate': 0.012642664173345464}. Best is trial 0 with value: 0.802536977905608.
[I 2022-05-22 03:21:07,579] Trial 1 finished with value: 0.8182510487644284 and parameters: {'n_estimators': 170, 'max_depth': 5, 'colsample_bytree': 0.35, 'learning_rate': 0.01617702786987782}. Best is trial 1 with value: 0.8182510487644284.
[I 2022-05-22 03:21:08,550] Trial 2 finished with value: 0.8361213010058048 and parameters: {'n_estimators': 150, 'max_depth': 8, 'colsample_bytree': 0.8, 'learning_rate': 0.04226790111175087}. Best is trial 2 with value: 0.8361213010058048.
[I 2022-05-22 03:21:10,415] Trial 3 finished with value: 0.8791450930109976 and parameters: {'n_estimators': 300, 'max_depth': 8, 'colsample_bytree': 0.75, 'le

In [33]:
xgb_params = xgb_study.best_params
opt_xgb_reg = xgb.XGBRegressor(n_estimators=xgb_params['n_estimators'],
                               max_depth=xgb_params['max_depth'],
                               colsample_bytree=xgb_params['colsample_bytree'],
                               learning_rate=xgb_params['learning_rate'])

opt_xgb_reg.fit(x_train, y_train)
opt_xgb_reg.score(x_test, y_test)


0.9315757134544769

LightGBM


In [37]:
def lgbm_reg_objective(trial: Trial):
    lgb_params =  {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 7, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda',0, 10)
    }

    lgbm_reg = lgb.LGBMRegressor(**lgb_params)
    cv = KFold(n_splits=5, shuffle=True)
    scores = cross_val_score(lgbm_reg, x_train, y_train, cv=cv, scoring='r2')
    return scores.mean()


lgbm_study = optuna.create_study(sampler=optuna.samplers.RandomSampler(), direction='maximize')
lgbm_study.optimize(lgbm_reg_objective, n_trials=300)


[I 2022-05-22 03:29:05,015] A new study created in memory with name: no-name-5ffb0d69-d756-48ff-b472-b2b457f33805
[I 2022-05-22 03:29:05,347] Trial 0 finished with value: 0.6677534713340125 and parameters: {'n_estimators': 149, 'num_leaves': 10, 'min_child_samples': 21, 'learning_rate': 0.9890946088557852, 'subsample': 0.6356651986252481, 'subsample_freq': 1, 'colsample_bytree': 0.6051082262442511, 'reg_alpha': 5.602146495540206, 'reg_lambda': 7.3831931626986105}. Best is trial 0 with value: 0.6677534713340125.
[I 2022-05-22 03:29:07,605] Trial 1 finished with value: 0.8445647974803586 and parameters: {'n_estimators': 294, 'num_leaves': 65, 'min_child_samples': 9, 'learning_rate': 0.2590544604552447, 'subsample': 0.8867228009544865, 'subsample_freq': 1, 'colsample_bytree': 0.6357105556346443, 'reg_alpha': 7.319339595595181, 'reg_lambda': 3.8539209596665636}. Best is trial 1 with value: 0.8445647974803586.
[I 2022-05-22 03:29:07,795] Trial 2 finished with value: 0.814337337347536 and pa

In [39]:
lgb_params = lgbm_study.best_params

opt_lgbm_reg = lgb.LGBMRegressor(
    n_estimators = lgb_params['n_estimators'],
    num_leaves = lgb_params['num_leaves'],
    min_child_samples = lgb_params['min_child_samples'],
    learning_rate = lgb_params['learning_rate'],
    reg_lambda = lgb_params['reg_lambda'],
    reg_alpha = lgb_params['reg_alpha'],
    colsample_bytree = lgb_params['colsample_bytree'],
    subsample_freq = lgb_params['subsample_freq'],
    subsample = lgb_params['subsample'],
    n_jobs = -1,
    random_state = 42,
)

opt_lgbm_reg.fit(x_train, y_train)
opt_lgbm_reg.score(x_test, y_test)


0.9285096782453484

In [40]:
opt_score = []
opt_model = []
opt_score.append(opt_rdf_reg.score(x_test, y_test))
opt_model.append('opt_rdf')

opt_score.append(opt_lgbm_reg.score(x_test, y_test))
opt_model.append('opt_lgbm')

opt_score.append(opt_xgb_reg.score(x_test, y_test))
opt_model.append('opt_xgb')


In [41]:
final_scores = pd.DataFrame()
final_scores['model_name'] = opt_model
final_scores['score_test'] = opt_score
final_scores = final_scores.sort_values(by=['score_test'], ascending=0)
final_scores

,model_name,score_test
0,opt_rdf,0.936014
2,opt_xgb,0.931576
1,opt_lgbm,0.928510


In [42]:
import joblib
joblib.dump(opt_rdf_reg, 'opt_rdf_reg.pkl')
joblib.dump(opt_xgb_reg, 'opt_xgb_reg.pkl')
joblib.dump(opt_lgbm_reg, 'opt_lgbm_reg.pkl')

['opt_lgbm_reg.pkl']